In [ ]:
# Remove all mentions of blouse_DF and sunderas before writing to CSV, directly after making dataframe
# Another approach is to remove the JSON files of everything not MORIA.... and the MIROA with blouse_DF etc.
import json
import pandas as pd
from pandas.io.json import json_normalize 
from pandasql import sqldf
import numpy as np
import os
from os import listdir
from os.path import isfile, join
DATAPATH = "../../data/Datasets_Enhancy/Annotation/"
IMAGEPATH = "../../data/Datasets_Enhancy/datasets/"

onlyfiles = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]
ret = []
for f in onlyfiles:
    #if ("MIROA") in f and not ("blouse" in f or "sudaderas" in f or "top_DF" in f):
    ret.append(f)
print(ret)
frames =[]
for i in range(len(ret)):
    with open(DATAPATH + ret[i]) as f:
        d = json.load(f)
        frames.append(pd.json_normalize(d['annotations']))
        
df = pd.concat(frames)
exportDf = df[[i for i in df.columns if "image_id" in i or "attributes" in i]]
exportDf

In [ ]:
deleted = 0
changed = 0
already_dropped = 0
s = len(exportDf.index)
outPut = exportDf.copy()

for i in range(s):
    id = exportDf.image_id.iloc[i]
    path1 = IMAGEPATH + id
    path2 = IMAGEPATH + "dataset/" + id
    path3 = IMAGEPATH + "Luxury_shops/" + id

    
    if not os.path.isfile(path1):
        if not os.path.isfile(path2):
            if not os.path.isfile(path3):
                try:
                    idx = outPut.loc[outPut['image_id'] == id].index[0]
                    print("DROPPED AT idx {} path {}".format(i, path2))
                    outPut = outPut.drop(idx, axis = 0)
                    deleted += 1
                except:
                    print("ALREADY DROPPED {}".format(path2))
                    already_dropped += 1
            
newLen = len(outPut.index)
for i in range(newLen):
    id = outPut.image_id.iloc[i]
    if os.path.isfile(IMAGEPATH + "dataset/" + id):
        outPut.loc[outPut['image_id'] == id, "image_id"] = "dataset/" + id
    elif os.path.isfile(IMAGEPATH + "Luxury_shops/" + id):
        outPut.loc[outPut['image_id'] == id, "image_id"] = "Luxury_shops/" + id

outPut["id"] = np.arange(len(outPut))
print("Total: {}".format(s))    
print("Deleted: {}".format(deleted))
print("Already Deleted: {}".format(already_dropped))
print("Edited: {}".format(changed))
#outPut.to_csv(r'./export_data.csv', index = False)
outPut

In [ ]:
# Create dicts which specifies valid entries for each type of attribute
# Create masterlist with all types of attributes
attribute_types = {}
attribute_map = {}
i = 0
for col in cols:
    vals = df[col].drop_duplicates().tolist()
    attribute_types[col] = vals
    attribute_map[col] = i
    i += 1
print(attribute_map)
print(attribute_types)

In [ ]:
newLabel = []
for index, row in exportDf.iterrows():
    attr = row["attributes"]
    res = []
    for t in attribute_types:
        possible = attribute_types[t]
        for val in possible:
            print(val)
            if "Col_Classique" in val:
                print("TEST")
                break
            if not "Null" in val and val in attr:
                break
                res.append(val)
            else:
                res.append("Null")
    newLabel.append(res)
                
    break
print(newLabel)

In [ ]:
CSVPATH = r"../../data/csv/export_labels_working.csv"
outPut.to_csv(CSVPATH, index = False)
